In [ ]:

import sqlite3
conn = sqlite3.connect("AQI.db")
with conn:
    sql = '''
    CREATE TABLE IF NOT EXISTS records (
        id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
        sitename TEXT NOT NULL,
        county TEXT,
        aqi INTEGER,
        status TEXT,
        pm25 NUMERIC,
        date TEXT,
        lat NUMERIC,
        lon NUMERIC,
        UNIQUE(sitename,date)
    );
    '''
    cursor = conn.cursor()
    cursor.execute(sql)

In [ ]:
import requests

url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'
try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
except Exception as e:
    print(e)
else:
    sitenames = set()
    for items in data['records']:
        sitenames.add(items['sitename'])

    sitenames = list(sitenames)
print(sitenames)

In [ ]:
import datasource
datasource.get_selected_data(sitename='屏東(枋山)')

In [ ]:
import sqlite3
conn = sqlite3.connect("AQI.db")
with conn:
    sql = '''INSERT OR IGNORE INTO records(sitename,county,aqi,status,pm25,date,lat,lon)
    values ('屏東(枋山)','屏東縣', 34, '良好',18,'2024-11-04 10:00', 22.260899, 120.651472);
    '''
    cursor = conn.cursor()
    cursor.execute(sql)

In [ ]:
import sqlite3
conn = sqlite3.connect("AQI.db")
url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'
try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
except Exception as e:
    print(e)
else:
    sitenames = set()
    with conn:
        cursor = conn.cursor()
        for items in data['records']:
            sitename = items['sitename']
            county = items['county']
            aqi = int(items['aqi']) if items['aqi'] != '' else 0
            status = items['status']
            pm25 = float(items['pm2.5']) if items['pm2.5'] != '' else 0.0
            date = items['datacreationdate']
            lon = float(items['longitude']) if items['longitude'] != '' else 0.0
            lat = float(items['latitude']) if items['latitude'] != '' else 0.0
            sql = '''INSERT OR IGNORE INTO records(sitename,county,aqi,status,pm25,date,lat,lon)
                    values (?,?, ?, ?,?,?,?,?);
            '''
            cursor.execute(sql,(sitename, county, aqi, status,pm25,date,lat,lon))

        